# Optimizing spin squeezing parameter readout
_Ilija Nikolov, Will Kaufman, December 2020_

In [ ]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import binom
from scipy.optimize import minimize
from scipy.interpolate import interp1d

In [ ]:
I=7/2
(Ix, Iy, Iz) = qt.spin_J_set(I)

In [ ]:
#The constants for our system
omegaQ = 4500 * 2*np.pi #7.58*2*np.pi*1e3
omegaL = 65.598*2*np.pi*1e6
omegaT = 1.58*2*np.pi*1e3
eta = omegaT/omegaQ

In [ ]:
Hsys = omegaQ * Iz**2 / 2  # ignoring eta term for now

In [ ]:
theta = np.pi/2
phi = np.pi

Calculate initial coherent state

In [ ]:
vals = np.round(Iz.eigenstates()[0], 1)
vects = Iz.eigenstates()[1]
state = vects[0] - vects[0]
for i in range(len(vals)):
    state = state + np.sqrt(binom(2*I, I+vals[i])) *\
    (np.cos(theta/2))**(I-vals[i])*(np.sin(theta/2))**(I+vals[i]) *\
    np.exp(-1j*(I+vals[i])*phi)*vects[i]

ksi = state * state.dag()

In [ ]:
initial_Ix = qt.expect(Ix, ksi)

In [ ]:
times = np.arange(0, 1000e-6, 5e-6)

e_ops = [
    Ix, Iy, Iz,
    Iz**2-Iy**2,Iz*Iy+Iy*Iz, Iz**2+Iy**2
]
e_names = [
    r'$I_x$', r'$I_y$', r'$I_z$',
    r'$I_z^2 - I_y^2$', r'$I_zI_y + I_yI_z$', r'$I_z^2 + I_y^2$'
]
result = qt.mesolve(Hsys, ksi, times, e_ops=e_ops)

In [ ]:
# for i, name in enumerate(e_names[:3]):
#     plt.plot(times, result.expect[i], label=name)
# plt.legend()
# for i, name in enumerate(e_names[3:]):
#     plt.plot(times, result.expect[i], label=name)
# plt.legend()

Calculate the squeezing parameter from `result` measurements.

In [ ]:
xi = (
    np.sqrt(result.expect[5] - np.sqrt(result.expect[3]**2 + result.expect[4]**2))
    / np.sqrt(I)
)

In [ ]:
plt.plot(times, xi, label=r'$\xi$')
plt.legend()
plt.xlabel('time (s)')

**Question**: as can be seen in the plot below, for $I=3/2$ $\xi$ seems to have a parabolic-like relationship with $I_x$. In the $I=1$ case, the relationship appears to be $\xi \propto |I_x|$ (you can re-run the code above to see what that looks like). Whatever the relationship between $\xi$ and $I_x$ is, **as long as it's a single-valued function (passes the "vertical line test"), couldn't you measure $I_x$ then estimate $\xi$ based on a curve fit**?

In [ ]:
fit = interp1d(result.expect[0] / initial_Ix, xi, kind='cubic')

In [ ]:
normalized_Ix = result.expect[0] / initial_Ix
plt.scatter(normalized_Ix, xi)

x = np.linspace(np.min(normalized_Ix), np.max(normalized_Ix), 100)
plt.plot(x, fit(x), color='orange')

plt.xlabel(r'$\langle I_x \rangle / \langle I_x \rangle_0$')
plt.ylabel(r'$\xi$')

In [ ]:
fit(.5)

If you can simulate the relationship between $\xi$ and $I_x$ in QuTiP, then (if it's single-valued) use that to estimate $\xi$ from experimental data, that seems like the best option.

Below I tried to think about unitary transformations that will make a better parabolic fit between $\xi$ and $I_x$. but if you let the curve fit be general enough I don't think it's necessary to apply the transformation at all.

I looked at the $\xi$-$I_x$ relationship for different spins (3/2 through 7/2), and they all seem to have that single-valued relationship.